## Scraping module
Scraping of straits times news article using selenium that are returned from the search function (keyword - 'food supply')

### Imports

In [3]:
#!pip install nltk
#import nltk
#nltk.download('punkt')

In [4]:
#!pip install newspaper3k

In [5]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import time

from newspaper import Article
import pandas as pd

### Newspaper Article Summariser

In [6]:
def getSummary(url):
    article = Article(url)
    article.download()
    article.parse()
    article.nlp()
    return article.summary, article.keywords

### StraitsTimes Scraper

In [7]:
driver = webdriver.Chrome(r"C:\Users\jonat\chromedriver\chromedriver.exe")
def getNews():
    output = []
    
    # get news title in a page - need a for loop to get all the pages!
    all_news_per_page=WebDriverWait(driver,20).until(EC.visibility_of_all_elements_located((By.CLASS_NAME,"queryly_item_title")))
    
    # get news url (i.e. news source) & news summary & keywords using the summariser module
    for i in range(len(all_news_per_page)):
        news_details = []
        news_details.append(all_news_per_page[i].text)
        xpath = f'//*[@id="resultdata"]/div[{i+2}]/a'        
        url = all_news_per_page[i].find_element_by_xpath(xpath).get_attribute('href')
        news_details.append(url)
        summary = getSummary(url)
        news_details.append(summary[0])
        news_details.append(", ".join(summary[1]))
        output.append(news_details)
    return output

C:\Users\jonat\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  """Entry point for launching an IPython kernel.


In [13]:
def getStraitsTimesNews(searchTerm):
    base_url = "https://www.straitstimes.com/search?searchkey=" + searchTerm.replace(" ", "%20")
    driver.get(base_url)

    # close ad (if any)
    try:
        iframe=driver.find_elements_by_tag_name("iframe")[3]
        driver.switch_to.frame(iframe)
        driver.find_elements_by_tag_name("iframe")
        ad = driver.switch_to.frame(0)
        driver.find_element_by_id('pclose-btn').click()
        driver.switch_to.default_content()
    except:
        driver.switch_to.default_content()
    
    # filter the search result by date by xpath accordingly
    # within 24 hours, within one week, within 30 days, within a year, within three years
    filter_by_date = WebDriverWait(driver,20).until(EC.visibility_of_all_elements_located((By.XPATH,"//*[contains(text(), 'within a year')]")))[0].click()

    # get news on page
    output = getNews()
    
    # check for next page
    nextpage = True
    while (nextpage):  
        try:
            nextpage = WebDriverWait(driver,20).until(EC.visibility_of_all_elements_located((By.CSS_SELECTOR,'#resultdata > a > h2')))[0]
            if (nextpage.text == 'Next Page'):
                nextpage.click()
#                 print('clicked next')
                output.extend(getNews())
            else: 
                nextpage = False
        except:
#             print("no next page!")
            nextpage = False
    
    
    # convert to df and export as csv
    df = pd.DataFrame(output, columns=['news title', 'news source(url)', 'content summary', 'keywords'])
    df.to_csv(f"csv_data/{searchTerm}_straitstimes.csv")
    return df

getStraitsTimesNews("farm")

C:\Users\jonat\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  import sys
C:\Users\jonat\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  if __name__ == '__main__':
C:\Users\jonat\Anaconda3\lib\site-packages\selenium\webdriver\remote\webelement.py:393: UserWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  warnings.warn("find_element_by_* commands are deprecated. Please use find_element() instead")


,news title,news source(url),content summary,keywords
0,Start-ups bringing Pakistan's farming into dig...,https://www.straitstimes.com/asia/south-asia/s...,"CHAK 26 SP, Pakistan (AFP) - Agriculture entre...","farming, way, project, 26, farmers, mr, told, ..."
1,Worm your way into an insect farm,https://www.straitstimes.com/life/travel/worm-...,SINGAPORE - Having been squeamish about insect...,"way, superwormshundreds, farm, worm, yelp, squ..."
2,New high-tech trout farm to boost Singapore's ...,https://www.straitstimes.com/singapore/new-hig...,SINGAPORE - A high-tech farm cultivating trout...,"farmed, production, farm, singapore, trout, dr..."
3,China to curb purchases of US farm goods in 20...,https://www.straitstimes.com/business/economy/...,China may continue to limit soya bean and othe...,"prices, 2022, solutions, ties, china, risk, pu..."
4,"Food supply hit ahead of CNY, Ramadan after fl...",https://www.straitstimes.com/asia/se-asia/food...,"""I had to throw away some 1,200 eggs as they c...","cny, supply, farm, malaysian, floods, told, wi..."
...,...,...,...,...
95,Eating Air: Discover the farm life in New Zealand,https://www.straitstimes.com/life/eating-air-d...,So my recent Sunday morning virtual trip to a ...,"chin, farm, discover, air, wool, screen, named..."
96,"Young Indonesians turn to farming, offering ho...",https://www.straitstimes.com/asia/se-asia/youn...,JAKARTA - Some young Indonesians are abandonin...,"farming, spot, technology, countrys, indonesia..."
97,"Caged animal farming must end in EU, European ...",https://www.straitstimes.com/world/europe/cage...,BRUSSELS (REUTERS) - The European Commission s...,"farming, end, european, eu, animals, cages, ci..."
98,'Mad' Israeli quest to revive ancient dates be...,https://www.straitstimes.com/world/middle-east...,'A lot of hassling'Before embarking on her dat...,"ancient, israeli, bears, palm, fruit, solowey,..."
